# Download and extract dataset

In [ ]:
# Download dataset from Google Drive

import urllib.request

url = 'https://drive.usercontent.google.com/download?id=1lhAaeQCmk2y440PmagA0KmIVBIysVMwu&export=download&authuser=0&confirm=t&uuid=3077628e-fc9b-4ef2-8cde-b291040afb30&at=APZUnTU9lSikCSe3NqbxV5MVad5T%3A1708243355040'
filename = 'tennis_court_det_dataset.zip'
urllib.request.urlretrieve(url, filename)

In [ ]:
# Unzip dataset

import zipfile

with zipfile.ZipFile("tennis_court_det_dataset.zip", "r") as zip_ref:
    zip_ref.extractall("tennis_court_det_dataset")


# Start Code

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms

import json
import cv2
import numpy as np

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Create Torch Dataset

In [3]:
class KeypointsDataset(Dataset):
    def __init__(self, img_dir, data_file):
        self.img_dir = img_dir
        with open(data_file, "r") as file:
            self.data = json.load(file)
        
        # Transform images to standardize them
        self.transforms = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    
    # Define length of dataset
    def __len__(self):
        return len(self.data)
    
    # Read items one by one and return the image and keypoints
    def __getitem__(self, idx):
        item = self.data[idx]
        img = cv2.imread(f"{self.img_dir}/{item['id']}.png")
        h,w = img.shape[:2]

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transforms(img)
        kps = np.array(item['kps']).flatten()
        kps = kps.astype(np.float32)

        kps[::2] *= 224.0 / w # Adjust x coordinates
        kps[1::2] *= 224.0 / h # Adjust y coordinates

        return img, kps

In [4]:
train_dataset = KeypointsDataset("tennis_court_det_dataset/data/images","tennis_court_det_dataset/data/data_train.json")
val_dataset = KeypointsDataset("tennis_court_det_dataset/data/images","tennis_court_det_dataset/data/data_val.json")

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)

### Create model

In [5]:
model = models.resnet50(pretrained=True)
model.fc =  torch.nn.Linear(model.fc.in_features, 14*2) # Replaces the last layer

c:\Users\Alex\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Alex\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
model = model.to(device)

### Train model

In [7]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4) # Can modify learning rate

In [8]:
epochs=20
for epoch in range(epochs):
    for i, (imgs,kps) in enumerate(train_loader):
        imgs = imgs.to(device)
        kps = kps.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, kps)
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print(f"Epoch {epoch}, iter {i}, loss: {loss.item()}")

Epoch 0, iter 0, loss: 14855.0751953125
Epoch 0, iter 10, loss: 15349.0341796875
Epoch 0, iter 20, loss: 14636.1171875
Epoch 0, iter 30, loss: 13781.810546875
Epoch 0, iter 40, loss: 13333.6298828125
Epoch 0, iter 50, loss: 13230.0205078125
Epoch 0, iter 60, loss: 13219.6259765625
Epoch 0, iter 70, loss: 11840.63671875
Epoch 0, iter 80, loss: 12086.0947265625
Epoch 0, iter 90, loss: 11329.697265625
Epoch 0, iter 100, loss: 11573.4306640625
Epoch 0, iter 110, loss: 10396.8486328125
Epoch 0, iter 120, loss: 10546.208984375
Epoch 0, iter 130, loss: 10034.84375
Epoch 0, iter 140, loss: 10092.1953125
Epoch 0, iter 150, loss: 9183.181640625
Epoch 0, iter 160, loss: 9397.6904296875
Epoch 0, iter 170, loss: 8688.029296875
Epoch 0, iter 180, loss: 8319.0927734375
Epoch 0, iter 190, loss: 8406.416015625
Epoch 0, iter 200, loss: 7571.37890625
Epoch 0, iter 210, loss: 7449.2080078125
Epoch 0, iter 220, loss: 7235.92578125
Epoch 0, iter 230, loss: 6701.69091796875
Epoch 0, iter 240, loss: 6441.7905

In [9]:
# Save predicted keypoints
torch.save(model.state_dict(), "keypoints_model.pth")